## LiteLLM <> Guardrails
* LiteLLM supports multiple LLM providers in one package. This notebook shows how you can use the [Getting Started example](https://www.guardrailsai.com/docs/guardrails_ai/getting_started) with LiteLLM.
* For your use case, all you need to do is the following:
    - Set the correct API key for your provider
    - Use the correct `MODEL_NAME` for your provider
* For more info, please refer to the [LiteLLM documentation](https://docs.litellm.ai/docs/)
* In this example, we will use OpenAI as the provider and the `gpt-3.5-turbo` model.

In [1]:
import os

# Set your API key
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

# Set the name of the model
MODEL_NAME = "gpt-3.5-turbo"

### Step 1: Install all dependencies

In [2]:
! pip install -U guardrails-ai litellm openai -q

### Step 2: Define the prompt and the Pydantic model with the validators

In [4]:
doctors_notes = """49 y/o Male with chronic macular rash to face & hair, worse in beard, eyebrows & nares.
Itchy, flaky, slightly scaly. Moderate response to OTC steroid cream"""

In [5]:
from pydantic import BaseModel, Field
from typing import List
from guardrails.validators import ValidRange, ValidChoices

prompt = """
Given the following doctor's notes about a patient, please extract a dictionary that contains the patient's information.

${doctors_notes}

${gr.complete_json_suffix_v2}
"""

class Symptom(BaseModel):
    symptom: str = Field(description="Symptom that a patient is experiencing")
    affected_area: str = Field(description="What part of the body the symptom is affecting", validators=[ValidChoices(choices=['head', 'neck', 'chest'], on_fail="reask")])

class Medication(BaseModel):
    medication: str = Field(description="Name of the medication the patient is taking")
    response: str = Field(description="How the patient is responding to the medication")


class PatientInfo(BaseModel):
    gender: str = Field(description="Patient's gender")
    age: int = Field(validators=[ValidRange(min=0, max=100, on_fail="fix")])
    symptoms: List[Symptom] = Field(description="Symptoms that the patient is currently experiencing. Each symptom should be classified into a separate item in the list.")
    current_meds: List[Medication] = Field(description="Medications the patient is currently taking and their response")

### Step 3: Create the `guard` object with the defined schema and the prompt

In [6]:
import guardrails as gd

# From pydantic:
guard = gd.Guard.from_pydantic(output_class=PatientInfo, prompt=prompt)

### Step 4: Make the `guard` call
* Use `litellm.completion` as the first argument
* Use the correct `MODEL_NAME` for the arg: `model`

In [7]:
import litellm
import json

# Wrap the OpenAI API call with the `guard` object
res = guard(
    litellm.completion,
    model=MODEL_NAME,
    prompt_params={"doctors_notes": doctors_notes},
    max_tokens=1024,
    temperature=0.3,
)

# Print the validated output from the LLM
print(json.dumps(res.validated_output, indent=2))

/Users/karanacharya/.pyenv/versions/3.11.6/envs/oss/lib/python3.11/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_max_budget" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(

Diffusion not supported. Skipping import.
11:31:11 - LiteLLM:INFO: 

POST Request Sent from LiteLLM:
curl -X POST \
https://api.openai.com/v1/ \
-d '{'model': 'gpt-3.5-turbo', 'messages': [{'role': 'user', 'content': '\nGiven the following doctor\'s notes about a patient, please extract a dictionary that contains the patient\'s information.\n\n49 y/o Male with chronic macular rash to face & hair, worse in beard, eyebrows & nares.\nItchy, flaky, slightly scaly. Moderate response to OTC steroid cream\n\n\nGiven below is XML that describes the information to extract from this document and the tags to extract it into.\n\n<output>\n    <string name="gender" description="Patient\'


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
11:31:15 - LiteLLM:INFO: Wrapper: Completed Call, calling success_handler
Wrapper: Completed Call, calling success_handler
11:31:15 - LiteLLM:INFO: 

POST Request Sent from LiteLLM:
curl -X POST \
https://api.openai.com/v1/ \
-d '{'model': 'gpt-3.5-turbo', 'messages': [{'role': 'user', 'content': '\nI was given the following JSON response, which had problems due to incorrect values.\n\n{\n  "gender": "Male",\n  "age": 49,\n  "symptoms": [\n    {\n      "symptom": "Chronic macular rash",\n      "affected_area": {\n        "incorrect_value": "face",\n        "error_messages": [\n          "Value face is not in choices [\'head\', \'neck\', \'chest\']."\n        ]\n      }\n    },\n    {\n      "symptom": "Chronic macular rash",\n      "affected_area": {\n        "incorrect_value": "hair",\n        "error_messages": [\n          "Value hair is not in choices [\'head\', \'neck\', \'chest\']."\n        ]\n      }


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
11:31:19 - LiteLLM:INFO: Wrapper: Completed Call, calling success_handler
Wrapper: Completed Call, calling success_handler


{
  "gender": "Male",
  "age": 49,
  "symptoms": [
    {
      "symptom": "Chronic macular rash",
      "affected_area": "head"
    },
    {
      "symptom": "Chronic macular rash",
      "affected_area": "head"
    },
    {
      "symptom": "Itchy",
      "affected_area": "head"
    },
    {
      "symptom": "Flaky",
      "affected_area": "head"
    },
    {
      "symptom": "Slightly scaly",
      "affected_area": "head"
    }
  ],
  "current_meds": [
    {
      "medication": "OTC steroid cream",
      "response": "Moderate response"
    }
  ]
}


### Step 5: See the entire history of calls

In [8]:
from rich import print
print(guard.history.last.tree)

Logs
├── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
│   │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
│   │ │                                                                                                         │ │
│   │ │ Given the following doctor's notes about a patient, please extract a dictionary that contains the       │ │
│   │ │ patient's information.                                                                                  │ │
│   │ │                                                                                                         │ │
│   │ │ 49 y/o Male with chronic macular rash to face & hair, worse in beard, eyebrows & nares.                 │ │
│   │ │ Itchy, flaky, slightly scaly. Moderate response to OTC steroid cream                                    │ │
│   │ │                                                                                                         │ │
│   │ │                                                                                                         │ │
│   │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
│   │ │ it into.                                                                                                │ │
│   │ │                                                                                                         │ │
│   │ │ <output>                                                                                                │ │
│   │ │     <string name="gender" description="Patient's gender"/>                                              │ │
│   │ │     <integer name="age" format="valid-range: min=0 max=100"/>                                           │ │
│   │ │     <list name="symptoms" description="Symptoms that the patient is currently experiencing. Each        │ │
│   │ │ symptom should be classified into a separate item in the list.">                                        │ │
│   │ │         <object>                                                                                        │ │
│   │ │             <string name="symptom" description="Symptom that a patient is experiencing"/>               │ │
│   │ │             <string name="affected_area" description="What part of the body the symptom is affecting"   │ │
│   │ │ format="valid-choices: choices=['head', 'neck', 'chest']"/>                                             │ │
│   │ │         </object>                                                                                       │ │
│   │ │     </list>                                                                                             │ │
│   │ │     <list name="current_meds" description="Medications the patient is currently taking and their        │ │
│   │ │ response">                                                                                              │ │
│   │ │         <object>                                                                                        │ │
│   │ │             <string name="medication" description="Name of the medication the patient is taking"/>      │ │
│   │ │             <string name="response" description="How the patient is responding to the medication"/>     │ │
│   │ │         </object>                                                                                       │ │
│   │ │     </list>                                                                                             │ │
│   │ │ </output>                                                                                               │ │
│   │ │                                                                                                         │ │
│   │ │                                                                                                         │ │
│   │ │ ONLY return a valid JSON object (no other t

### Conclusion
This is a simple example to show how you can use LiteLLM with Guardrails. For more complex use cases, please refer to the [Guardrails documentation](https://www.guardrailsai.com/docs/guardrails_ai/getting_started) and the [LiteLLM documentation](https://docs.litellm.ai/docs/) and set up your workflow accordingly.